In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
except:
    print("Using Local Machine")

In [1]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...



[notice] A new release of pip is available: 23.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from yolov5.models.yolo import Model
from yolov5.utils.dataloaders import LoadImagesAndLabels
from yolov5.utils.general import check_requirements, set_logging
from yolov5.utils.loss import ComputeLoss
from yolov5.utils.plots import plot_labels, plot_results
from pathlib import Path
import yaml



In [2]:
import os
import random
from pathlib import Path
import pandas as pd
import cv2


In [7]:
def ConvertLisaToYolo(dataSetFolderPath: str,outputPath: str  ):
    # annotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    uniqueFiles = annotationsDataFrame['Filename'].unique()
    for file in uniqueFiles:
        image_file = os.path.join(dataSetFolderPath, file)
        if not os.path.exists(image_file):
            continue
        fileDataFrame = annotationsDataFrame[annotationsDataFrame['Filename'] == file]
        fileAnnotations = []
        for index, row in fileDataFrame.iterrows():
            x1, y1, x2, y2 = row[2], row[3], row[4], row[5]
            image = cv2.imread(dataSetFolderPath+"/"+row[0])
            h, w, _ = image.shape
            xCenter = ((x1 + x2) / 2) / w
            yCenter = ((y1 + y2) / 2) / h
            width = (x2 - x1) / w
            height = (y2 - y1) / h
            classId = uniqueSigns.index(row[1])
            fileAnnotations.append(f"{classId} {xCenter} {yCenter} {width} {height}")
        filepath = dataSetFolderPath+"/"+"/".join(file.split("/")[:-1])
        print(filepath)
        with open(os.path.join(filepath, f"{Path(file).stem}.txt"), "w") as f:
            f.write("\n".join(fileAnnotations))
def CreateTrainValFiles(dataSetFolderPath: str):
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    print(uniqueSigns)
    print(len(uniqueSigns))
    uniqueFiles = annotationsDataFrame['Filename'].unique().tolist()
    image_files = ["./DataSet/" + file for file in uniqueFiles ]
    random.shuffle(image_files)

    split_ratio = 0.8
    train_files = image_files[:int(len(image_files) * split_ratio)]
    val_files = image_files[int(len(image_files) * split_ratio):]

    with open("train.txt", "w") as f:
        f.writelines([ file + "\n" for file in train_files])

    with open("val.txt", "w") as f:
        f.writelines([file + "\n" for file in val_files])
    return len(uniqueSigns), uniqueSigns

def CreateYaml(fileName: str, numClass: int, classList: list):
    data = {
    'train': './train.txt',
    'val': './val.txt',
    'nc': numClass,
    'names': classList
    }

    with open(fileName, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

In [8]:
# os.mkdir('./Yolo Data Annotation')
ConvertLisaToYolo('./DataSet', './Yolo Data Annotation')
numClass, classList = CreateTrainValFiles('./DataSet')
CreateYaml('signboard.yaml', numClass, classList)

./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations
./DataSet/aiua120214-0/frameAnno

NameError: name 'yaml' is not defined

In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 100
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using {} device".format(device))


In [ ]:
# Load custom dataset configuration
with open('signboard.yaml') as f:
    data = yaml.safe_load(f)
train_path, val_path = data['train'], data['val']
nc, names = data['nc'], data['names']

In [ ]:
# Create output directory
save_dir = Path('runs/train/signboard_exp')
(save_dir / 'weights').mkdir(parents=True, exist_ok=True)
set_logging(save_dir / 'train.log')

In [ ]:
# Load dataset
train_dataset = LoadImagesAndLabels(train_path, 640, 16, rect=True, pad=0.5, cache=True)
val_dataset = LoadImagesAndLabels(val_path, 640, 16, rect=True, pad=0.5, cache=True)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)


In [ ]:
print("Downloading Weights of yolo5 Verion ", yolo5Version)
weightsURL = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(yolo5Version)
!wget {weightsURL}

In [ ]:
def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    # model.load_state_dict(torch.load("yolov5{}.pt".format(version))["model"].state_dict(), strict=False)

    return model


In [ ]:
# Load model
model = CreateYolov5Model(numClass,yolo5Version)
model.to(device)

In [ ]:
# Load pre-trained weights
ckpt = torch.load('yolov5s.pt', map_location=device)
ckpt['model'] = {k: v for k, v in ckpt['model'].items() if model.state_dict()[k].shape == v.shape}
model.load_state_dict(ckpt['model'], strict=False)


In [ ]:
# Initialize optimizer and loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.937, weight_decay=0.0005, nesterov=True)
criterion = ComputeLoss(model)

In [ ]:

epochs = 100
best_fitness = float('inf')

for epoch in range(epochs):
    model.train()

    for i, (imgs, targets, paths, _) in enumerate(train_loader):
        imgs = imgs.to(device).float() / 255.0
        targets = targets.to(device)

        # Forward pass
        pred = model(imgs)

        # Compute loss
        loss, loss_items = criterion(pred, targets)
        loss_items = torch.cat(loss_items)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

    # Validation
    model.eval()
    with torch.no_grad():
        for i, (imgs, targets, paths, _) in enumerate(val_loader):
            imgs = imgs.to(device).float() / 255.0
            targets = targets.to(device)

            # Forward pass
            pred = model(imgs)

            # Compute loss
            val_loss, val_loss_items = criterion(pred, targets)
            val_loss_items = torch.cat(val_loss_items)

    # Save best model
    if val_loss < best_fitness:
        best_fitness = val_loss
        torch.save(model.state_dict(), save_dir / 'weights' / 'best.pt')

# Plot training results
plot_results(save_dir=save_dir)
